In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np
import time
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.chart import BarChart, Reference

def scrape_psx_market_summary(chromedriver_path):
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")

    service = Service(executable_path=chromedriver_path)
    driver = webdriver.Chrome(service=service, options=options)
    driver.get("https://www.psx.com.pk/market-summary/")

    time.sleep(5)  # Wait for page to load fully, adjust if needed

    all_data = []

    # Find all tables (each sector)
    tables = driver.find_elements(By.TAG_NAME, "table")

    for table in tables:
        try:
            # Sector heading <h4> inside thead > th[colspan=8]
            sector_heading_th = table.find_element(By.CSS_SELECTOR, "thead > tr > th[colspan='8']")
            sector_name = sector_heading_th.find_element(By.TAG_NAME, "h4").text.strip()

            # Skip unwanted sectors
            if sector_name == "" or "Data refreshes" in sector_name or "* LDCP" in sector_name:
                continue

            tbody = table.find_element(By.TAG_NAME, "tbody")
            rows = tbody.find_elements(By.TAG_NAME, "tr")

            # First row is headers
            headers = [td.text.strip() for td in rows[0].find_elements(By.TAG_NAME, "td")]

            # Data rows
            for row in rows[1:]:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) != len(headers):
                    continue
                row_data = {headers[i]: cols[i].text.strip() for i in range(len(headers))}
                row_data['Sector'] = sector_name
                all_data.append(row_data)
        except Exception:
            # Ignore malformed tables
            continue

    driver.quit()

    df = pd.DataFrame(all_data)

    # Clean and convert numeric columns safely
    num_cols = ['LDCP', 'OPEN', 'HIGH', 'LOW', 'CURRENT', 'CHANGE', 'VOLUME']
    for col in num_cols:
        df[col] = df[col].astype(str).str.replace(',', '').replace('', np.nan)
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

def style_and_add_chart_to_excel(df, filename):
    df.to_excel(filename, index=False)

    wb = load_workbook(filename)
    ws = wb.active

    # Style header row: bold + background color + center align
    header_fill = PatternFill(start_color="FFC000", end_color="FFC000", fill_type="solid")
    header_font = Font(bold=True, color="FFFFFF")
    header_alignment = Alignment(horizontal="center")

    for cell in ws[1]:
        cell.fill = header_fill
        cell.font = header_font
        cell.alignment = header_alignment

    # Align numeric columns right, add thousand separator
    col_letter_map = {cell.value: cell.column_letter for cell in ws[1]}
    num_cols = ['LDCP', 'OPEN', 'HIGH', 'LOW', 'CURRENT', 'CHANGE', 'VOLUME']

    for col in num_cols:
        col_letter = col_letter_map.get(col)
        if not col_letter:
            continue
        for cell in ws[col_letter][1:]:  # skip header row
            cell.alignment = Alignment(horizontal='right')
            cell.number_format = '#,##0.00'

    # Add Bar Chart for top 10 CURRENT prices
    chart = BarChart()
    chart.title = "Top 10 Stocks by Current Price"
    chart.y_axis.title = 'Price'
    chart.x_axis.title = 'Stock SCRIP'

    top10 = df.sort_values('CURRENT', ascending=False).head(10)
    start_row = len(df) + 3

    # Write headers for chart data
    ws.cell(row=start_row, column=1, value='SCRIP')
    ws.cell(row=start_row, column=2, value='CURRENT')

    for i, (_, row) in enumerate(top10.iterrows(), start=start_row + 1):
        ws.cell(row=i, column=1, value=row['SCRIP'])
        ws.cell(row=i, column=2, value=row['CURRENT'])

    data_ref = Reference(ws, min_col=2, min_row=start_row, max_row=start_row + 10)
    cats_ref = Reference(ws, min_col=1, min_row=start_row + 1, max_row=start_row + 10)

    chart.add_data(data_ref, titles_from_data=True)
    chart.set_categories(cats_ref)

    ws.add_chart(chart, f"D{start_row}")

    wb.save(filename)
    print(f"Styled Excel with chart saved as '{filename}'")

def main():
    chromedriver_path = r'C:\chromedriver-win32\chromedriver.exe'  # Update path here
    print("Starting PSX market summary scraping...")
    df = scrape_psx_market_summary(chromedriver_path)
    print("Scraping done. Sample data:")
    print(df.head())

    output_excel = "psx_market_summary_styled.xlsx"
    style_and_add_chart_to_excel(df, output_excel)

if __name__ == "__main__":
    main()


Starting PSX market summary scraping...
Scraping done. Sample data:
                  SCRIP     LDCP     OPEN     HIGH      LOW  CURRENT  CHANGE  \
0     AL-Ghazi Tractors   388.58   386.11   427.44   386.11   427.44   38.86   
1       Atlas Honda Ltd  1149.99  1157.70  1165.00  1144.00  1144.98   -4.16   
2          Dewan Motors    37.25    37.48    37.48    36.20    36.29   -0.91   
3  Ghandhara Automobile   478.30   478.30   479.80   471.10   472.99   -5.83   
4        Ghandhara Ind.   695.78   695.00   699.00   685.00   685.99   -9.52   

    VOLUME                Sector  
0    63412  AUTOMOBILE ASSEMBLER  
1     6899  AUTOMOBILE ASSEMBLER  
2  1878172  AUTOMOBILE ASSEMBLER  
3   331180  AUTOMOBILE ASSEMBLER  
4   195793  AUTOMOBILE ASSEMBLER  
Styled Excel with chart saved as 'psx_market_summary_styled.xlsx'
